In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.stats import skew

from sklearn.linear_model import LogisticRegression , LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler,LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor, GradientBoostingClassifier, AdaBoostClassifier,StackingClassifier,ExtraTreesClassifier,VotingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import ensemble

from sklearn.metrics import accuracy_score,log_loss,make_scorer, mean_squared_error

import lightgbm as lgb

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVR
import warnings
warnings.simplefilter('ignore')
#赤背景になっている警告を非表示に

In [2]:
sample_submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
test_original = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
train_original = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train = train_original
test = test_original
y_train = train_original["Transported"]

data_origin=pd.concat([train,test])
data_origin.index = np.arange(0,12970)
data = data_origin
data[["PassengerGroup","PassengerNum"]] = data["PassengerId"].str.split("_",expand=True)

def split_column(value):
    if isinstance(value, str) and'/' in value:
        #isinstanceとは（）の最初のオブジェクト型と、2番目のオブジェクト型が一致しているか確認。今回はvalueがstrings型になっているかを確認
        return value.split('/')
    else:
        return [None, None,None]

# 列を分割して新しい列を作成
split_data = data["Cabin"].apply(split_column)
#dataframe.apply（関数）でdataframeに対して「関数」を繰り返し記録できる
data[["Cabin_1","Cabin_2","Cabin_3"]] = pd.DataFrame(split_data.tolist(), index=data.index)
data["Cabin_2"] = data["Cabin_2"].astype("float64")

In [3]:

# 欠損値の数を計算
missing_values_count = data.isnull().sum()

# 欠損値の数とパーセンテージを表形式で表示するためのDataFrameを作成
missing_values_table = pd.DataFrame({
    'Missing_total': missing_values_count,
    'Percent (%)': round((missing_values_count / len(data)) * 100, 1)
})

# 欠損値がある列の情報を抽出して降順でソート
df_missing = missing_values_table[missing_values_table['Missing_total'] > 0].sort_values(by='Missing_total', ascending=False)
    
# 欠損値がある列の情報を表示
display(df_missing)

,Missing_total,Percent (%)
Transported,4277,33.0
CryoSleep,310,2.4
ShoppingMall,306,2.4
Cabin,299,2.3
Cabin_1,299,2.3
Cabin_2,299,2.3
Cabin_3,299,2.3
VIP,296,2.3
Name,294,2.3
FoodCourt,289,2.2


In [4]:
"""
#VIPかそうでないかを分けて、欠損値処理を実施
list_0 = ["ShoppingMall","FoodCourt","Spa","VRDeck","RoomService"]

for vip_col in list_0:
    data[vip_col] = data[vip_col].mask(data[vip_col].isnull() & data["VIP"]==True,data.loc[data["VIP"]==True][vip_col].mean())

data[list_0] = data[list_0].fillna(0)
"""

'\n#VIPかそうでないかを分けて、欠損値処理を実施\nlist_0 = ["ShoppingMall","FoodCourt","Spa","VRDeck","RoomService"]\n\nfor vip_col in list_0:\n    data[vip_col] = data[vip_col].mask(data[vip_col].isnull() & data["VIP"]==True,data.loc[data["VIP"]==True][vip_col].mean())\n\ndata[list_0] = data[list_0].fillna(0)\n'

In [5]:
list_0 = ["ShoppingMall","FoodCourt","Spa","VRDeck","RoomService","Cabin_2"]
for col in list_0 :
    data[list_0] = data[list_0].fillna(0)

list_median = ["Age"]
data[list_median] = data[list_median].fillna(data[list_median].median())

list_None = ["Cabin_3","Cabin","Name"]
data[list_None] = data[list_None].fillna("None")
data["Cabin_1"] = data["Cabin_1"].fillna(0)

#list_node=["Destination","HomePlanet","VIP","CryoSleep"]
list_node=["Destination","HomePlanet"]
for col in list_node:
    mode_value = data[col].mode()[0]
    data[col] = data[col].fillna(mode_value)
#最頻値はdata.mode()をリスト形式で返すため、fillna(data.mode())では表示ができない。そのためfor文で入力している

list_None2 = ["VIP","CryoSleep"]
for col in list_None2:
    data[col] = data[col].fillna(2)

In [6]:
def torf(value):
    if value is False:
        return 0
    else:
        return 1
torf_list = ["CryoSleep","VIP"]

for col in  torf_list:
    data[col] = data[col].apply(torf)

In [7]:
"""
def overor0(value):
    if value!=0:
        return 1
    else:
        return 0

over0_list = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]    
for col in over0_list:
    data[col] = data[col].apply(overor0)
    
def cabin2(value):
    if value == "None":
        return "None"
    elif 0<=value<=300 or 750<value<=1150:
        return "0"
    else:
        return "1"
    
data["Cabin_2"] = data["Cabin_2"].apply(cabin2)

"""

'\ndef overor0(value):\n    if value!=0:\n        return 1\n    else:\n        return 0\n\nover0_list = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]    \nfor col in over0_list:\n    data[col] = data[col].apply(overor0)\n    \ndef cabin2(value):\n    if value == "None":\n        return "None"\n    elif 0<=value<=300 or 750<value<=1150:\n        return "0"\n    else:\n        return "1"\n    \ndata["Cabin_2"] = data["Cabin_2"].apply(cabin2)\n\n'

In [8]:
drop_idx = [8415,4416,5105,5722,7995,4311,5619]
#drop_idx=[725,3366,4311,5619,6547,1095,1390,1598,4278,5722,6921,7995,4724,6223,7425,8415,1213,1842,2067,3198,3538,928,1177,4416,4762,5105,5725,7118,7933,8626]
data = data.drop(drop_idx)

In [9]:
passengernum = data.groupby("PassengerGroup").count()["PassengerNum"]
data = pd.merge(data,passengernum,on="PassengerGroup",how="left")

data["pay_sum"] = data['RoomService']+data['FoodCourt']+data['ShoppingMall']+data['Spa']+data['VRDeck']

#data["Cabin_13"] = data["Cabin_1"].astype(str)+data["Cabin_3"]
data["Cabin_1"] = data["Cabin_1"].replace(["T","E","D","F","G","A","C","B"],np.arange(1,9))

In [10]:
"""
#グループの一人当たりの平均支払額を特徴量として追加
group_pay = data.groupby("PassengerGroup").sum()["pay_sum"]
data = pd.merge(data,group_pay,on="PassengerGroup",how="left")
data["pay_sum_y"] = data["pay_sum_y"]/data["PassengerNum_y"]
data = data.rename(columns={"pay_sum_x":"pay_sum","pay_sum_y":"pay_per_passenger"})
"""

'\n#グループの一人当たりの平均支払額を特徴量として追加\ngroup_pay = data.groupby("PassengerGroup").sum()["pay_sum"]\ndata = pd.merge(data,group_pay,on="PassengerGroup",how="left")\ndata["pay_sum_y"] = data["pay_sum_y"]/data["PassengerNum_y"]\ndata = data.rename(columns={"pay_sum_x":"pay_sum","pay_sum_y":"pay_per_passenger"})\n'

In [11]:
"""
name_list = data["Name"].str.split(" ").tolist()
data["Name_first"] = pd.DataFrame(list(map(lambda x:x[0],name_list)))
data["Name_last"] = pd.DataFrame(list(map(lambda x:x[1],name_list)))
#map(lambda x:xの処理 , list)で、listをxの処理へ繰り返すという関数になる。map はdefの代わり。mapはfor文の代わりになる

namelastnum = data.groupby(["Name_last","Cabin"]).count()["Name_first"]
data = pd.merge(data,namelastnum,on=["Name_last","Cabin"],how="left")
data = data.rename(columns={"Name_first_y":"Name_last_count"})
data
"""

'\nname_list = data["Name"].str.split(" ").tolist()\ndata["Name_first"] = pd.DataFrame(list(map(lambda x:x[0],name_list)))\ndata["Name_last"] = pd.DataFrame(list(map(lambda x:x[1],name_list)))\n#map(lambda x:xの処理 , list)で、listをxの処理へ繰り返すという関数になる。map はdefの代わり。mapはfor文の代わりになる\n\nnamelastnum = data.groupby(["Name_last","Cabin"]).count()["Name_first"]\ndata = pd.merge(data,namelastnum,on=["Name_last","Cabin"],how="left")\ndata = data.rename(columns={"Name_first_y":"Name_last_count"})\ndata\n'

In [12]:
"""
#歪度の削減の手法を用いていきます。そのため、まずは現状の歪度が高いもの (0.75以上) を抽出します。
from scipy.stats import skew
# 数値変数を特定
numeric_feats = data.dtypes[data.dtypes != "object"].index
# 各数値変数の歪度を計算し、降順にソート
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
# 歪度を含むデータフレームの作成
skewness = pd.DataFrame({'Skew' :skewed_feats})
# 歪度の絶対値が0.75より大きい特徴量を選択
skewness = skewness[abs(skewness) > 0.75]
# 歪度が0.75より小さい特徴量を削除
skewness = skewness.dropna(axis=0)


#歪度の絶対値が0.75以上のものに対してBox-Cox変換を適用していきます
from scipy.special import boxcox1p
# Box-Cox変換を適用する数値変数の列を特定
indices = skewness.index
# 新しい列を格納するための空のリストを作成
skewness_after_boxcox = []
# Box-Cox変換のパラメータ（ラムダ）を設定
lam = 0.15
# 各数値変数に対してBox-Cox変換を適用し、新しい列として追加
for idx in indices:
    transformed_values = boxcox1p(data[idx], lam)
    data[idx] = transformed_values
    # Box-Cox変換後の歪度を計算し、リストに追加
    skewness_after_boxcox.append(skew(transformed_values))
# skewnessデータフレームにBox-Cox変換後の歪度を追加
skewness['Skew_BoxCox'] = skewness_after_boxcox
# Box-Cox変換後の歪度を表示
display(skewness.head(10))
"""

'\n#歪度の削減の手法を用いていきます。そのため、まずは現状の歪度が高いもの (0.75以上) を抽出します。\nfrom scipy.stats import skew\n# 数値変数を特定\nnumeric_feats = data.dtypes[data.dtypes != "object"].index\n# 各数値変数の歪度を計算し、降順にソート\nskewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)\n# 歪度を含むデータフレームの作成\nskewness = pd.DataFrame({\'Skew\' :skewed_feats})\n# 歪度の絶対値が0.75より大きい特徴量を選択\nskewness = skewness[abs(skewness) > 0.75]\n# 歪度が0.75より小さい特徴量を削除\nskewness = skewness.dropna(axis=0)\n\n\n#歪度の絶対値が0.75以上のものに対してBox-Cox変換を適用していきます\nfrom scipy.special import boxcox1p\n# Box-Cox変換を適用する数値変数の列を特定\nindices = skewness.index\n# 新しい列を格納するための空のリストを作成\nskewness_after_boxcox = []\n# Box-Cox変換のパラメータ（ラムダ）を設定\nlam = 0.15\n# 各数値変数に対してBox-Cox変換を適用し、新しい列として追加\nfor idx in indices:\n    transformed_values = boxcox1p(data[idx], lam)\n    data[idx] = transformed_values\n    # Box-Cox変換後の歪度を計算し、リストに追加\n    skewness_after_boxcox.append(skew(transformed_values))\n# skewnessデータフレームにBox-Cox変換後の歪度を追加\nskewness[\'Skew_BoxCox\']

In [13]:
"""
data["HomePlanet"] = data["HomePlanet"].replace(["Europa","Earth","Mars"],[3,1,2])
data["Destination"] = data["Destination"].replace(["TRAPPIST-1e","55 Cancri e","PSO J318.5-22"],[1,3,2])

le = LabelEncoder()
data["Cabin_3"] = le.fit_transform(data["Cabin_3"])
"""

'\ndata["HomePlanet"] = data["HomePlanet"].replace(["Europa","Earth","Mars"],[3,1,2])\ndata["Destination"] = data["Destination"].replace(["TRAPPIST-1e","55 Cancri e","PSO J318.5-22"],[1,3,2])\n\nle = LabelEncoder()\ndata["Cabin_3"] = le.fit_transform(data["Cabin_3"])\n'

In [14]:
train_data = data
drop_list=["PassengerId","Cabin","Name","PassengerNum_x","PassengerGroup"]
train_data = train_data.drop(drop_list,axis=1)

#drop_list_a=["Cabin_1","Cabin_3"]
#train_data = train_data.drop(drop_list_a,axis=1)

In [15]:
data_trans=train_data["Transported"]
data_trans = data_trans.replace([True,False],[1,0])

data_obj = train_data.select_dtypes(include=object)
data_obj = data_obj.drop("Transported",axis=1)
data_obj_dummy = pd.get_dummies(data_obj,dtype=int)

data_category = train_data[['CryoSleep','VIP']]

In [16]:
"""
data_trans=train_data["Transported"]
data_trans = data_trans.replace([True,False],[1,0])

data_category = train_data[['CryoSleep','VIP']]
"""

'\ndata_trans=train_data["Transported"]\ndata_trans = data_trans.replace([True,False],[1,0])\n\ndata_category = train_data[[\'CryoSleep\',\'VIP\']]\n'

In [17]:
"""
data_number_sc = train_data[["Age","PassengerNum_y",'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck',"Cabin_1","Cabin_2","pay_sum"]]
columns = data_number.columns

sc = StandardScaler()
sc.fit(data_number)
data_std = sc.transform(data_number)
data_std = pd.DataFrame(data_std,columns=columns)
"""

'\ndata_number_sc = train_data[["Age","PassengerNum_y",\'RoomService\', \'FoodCourt\', \'ShoppingMall\',\'Spa\', \'VRDeck\',"Cabin_1","Cabin_2","pay_sum"]]\ncolumns = data_number.columns\n\nsc = StandardScaler()\nsc.fit(data_number)\ndata_std = sc.transform(data_number)\ndata_std = pd.DataFrame(data_std,columns=columns)\n'

In [18]:
data_number = train_data[["Age","PassengerNum_y",'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck',"Cabin_1","Cabin_2","pay_sum",]]
#data_number = train_data[["Age","PassengerNum_y",'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck',"Cabin_1","Cabin_2","pay_sum","HomePlanet","Destination","Cabin_3"]]
columns = data_number.columns

rs = RobustScaler()
#中央値を中心にスケーリングし、四分位範囲（IQR）を基準にスケーリングします。
#メリット:外れ値に対して頑健です。外れ値の影響を受けにくいため、外れ値が存在するデータセットで効果的です。
rs.fit(data_number)
data_std = rs.transform(data_number)
data_std = pd.DataFrame(data_std,columns=columns)

In [19]:
data_train = data_obj_dummy.join(data_std)
data_train = data_train.join(data_trans)
#data_train = data_std.join(data_trans)
data_train = data_train.join(data_category)
data_train

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Cabin_3_None,Cabin_3_P,Cabin_3_S,Age,...,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_1,Cabin_2,pay_sum,Transported,CryoSleep,VIP
0,0,1,0,0,0,1,0,1,0,0.705882,...,0.000000,0.000000,0.000000,0.000000,4.0,-0.490137,-0.496533,0.0,0,0
1,1,0,0,0,0,1,0,0,1,-0.176471,...,0.146341,1.086957,11.090909,1.222222,0.0,-0.490137,0.013870,1.0,0,0
2,0,1,0,0,0,1,0,0,1,1.823529,...,58.146341,0.000000,135.656566,1.361111,2.0,-0.490137,6.703883,0.0,0,1
3,0,1,0,0,0,1,0,0,1,0.352941,...,20.861789,16.130435,67.252525,5.361111,2.0,-0.490137,3.092926,0.0,0,0
4,1,0,0,0,0,1,0,0,1,-0.647059,...,1.138211,6.565217,11.414141,0.055556,0.0,-0.488942,0.260055,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12958,1,0,0,0,0,1,0,0,1,0.411765,...,0.000000,0.000000,0.000000,0.000000,1.0,1.298267,-0.496533,NaN,1,0
12959,1,0,0,0,0,1,1,0,0,0.882353,...,13.772358,0.739130,0.202020,4.000000,-4.0,-0.490137,0.209431,NaN,0,0
12960,0,0,1,1,0,0,0,1,0,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.0,-0.136282,-0.496533,NaN,1,0
12961,0,1,0,0,0,1,0,1,0,0.000000,...,43.577236,0.000000,0.000000,14.527778,-1.0,-0.135087,1.724688,NaN,0,0


In [20]:
"""
drop_list=['ShoppingMall',"Age",'HomePlanet_Earth','FoodCourt','Destination_TRAPPIST-1e', "Cabin_3_P"]
data_train = data_train.drop(drop_list,axis=1)
"""

'\ndrop_list=[\'ShoppingMall\',"Age",\'HomePlanet_Earth\',\'FoodCourt\',\'Destination_TRAPPIST-1e\', "Cabin_3_P"]\ndata_train = data_train.drop(drop_list,axis=1)\n'

In [21]:
df_train = data_train.loc[data_train["Transported"].notna()]
df_test = data_train.loc[data_train["Transported"].isna()]
df_test = df_test.drop("Transported",axis=1)

In [22]:
"""
from sklearn.feature_selection import SequentialFeatureSelector

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

model_lgbm = LGBMClassifier(verbose=-1)
sf = SequentialFeatureSelector(model_lgbm, scoring='accuracy', direction = 'backward')
sf.fit(X,y)

# 選ばれた特徴量のブールマスクを取得
selected_mask = sf.get_support()

# 選ばれたカラム名を取得
selected_columns = X.columns[selected_mask]

# 選ばれたカラム名を出力
print(f'Selected columns: {selected_columns}')

"""

'\nfrom sklearn.feature_selection import SequentialFeatureSelector\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nmodel_lgbm = LGBMClassifier(verbose=-1)\nsf = SequentialFeatureSelector(model_lgbm, scoring=\'accuracy\', direction = \'backward\')\nsf.fit(X,y)\n\n# 選ばれた特徴量のブールマスクを取得\nselected_mask = sf.get_support()\n\n# 選ばれたカラム名を取得\nselected_columns = X.columns[selected_mask]\n\n# 選ばれたカラム名を出力\nprint(f\'Selected columns: {selected_columns}\')\n\n'

In [23]:
selected_columns = ['HomePlanet_Europa', 'Destination_TRAPPIST-1e', 'Cabin_3_P',
       'PassengerNum_y', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Cabin_1', 'pay_sum']

In [24]:
"""
#isolation forestによる異常値検知
#Random Forestと同様に決定木に基づいて構築されます。決定木を各データが孤立するまで分割を繰り返し、データが孤立するまでの距離（深さ）から異常値を推定しよう、というのが基本的なアイディアです


from sklearn.ensemble import IsolationForest

# Isolation Forestによる外れ値検出
iso = IsolationForest(contamination=0.01)
yhat = iso.fit_predict(df_train)

# 外れ値を除外
mask = yhat != -1
df_train = df_train[mask]
"""

'\n#isolation forestによる異常値検知\n#Random Forestと同様に決定木に基づいて構築されます。決定木を各データが孤立するまで分割を繰り返し、データが孤立するまでの距離（深さ）から異常値を推定しよう、というのが基本的なアイディアです\n\n\nfrom sklearn.ensemble import IsolationForest\n\n# Isolation Forestによる外れ値検出\niso = IsolationForest(contamination=0.01)\nyhat = iso.fit_predict(df_train)\n\n# 外れ値を除外\nmask = yhat != -1\ndf_train = df_train[mask]\n'

In [25]:
"""
df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ベースモデルの定義
base_models = [
    ('knn', KNeighborsClassifier()),
    ('lr', LogisticRegression(max_iter=1000)),
    ('svc', SVC(probability=True)),
    ('rf', RandomForestClassifier()),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier(verbose=-1))
]

# メタモデルの定義
meta_model = LogisticRegression()

# Stacking Classifierの定義
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
# Stacking Classifierの学習
stacking.fit(X_train, y_train)
# 予測
y_pred = stacking.predict(X_test)
# 精度の評価
accuracy = accuracy_score(y_test, y_pred)
print(f'Stacking Model Accuracy: {accuracy:.4f}')


submit = stacking.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)
"""

'\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# ベースモデルの定義\nbase_models = [\n    (\'knn\', KNeighborsClassifier()),\n    (\'lr\', LogisticRegression(max_iter=1000)),\n    (\'svc\', SVC(probability=True)),\n    (\'rf\', RandomForestClassifier()),\n    (\'xgb\', XGBClassifier()),\n    (\'lgbm\', LGBMClassifier(verbose=-1))\n]\n\n# メタモデルの定義\nmeta_model = LogisticRegression()\n\n# Stacking Classifierの定義\nstacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)\n# Stacking Classifierの学習\nstacking.fit(X_train, y_train)\n# 予測\ny_pred = stacking.predict(X_test)\n# 精度の評価\naccuracy = accuracy_score(y_test, y_pred)\nprint(f\'Stacking Model Accuracy: {accuracy:.4f}\')\n\n\nsubmit = stacking.predict(df_test)\nsubmit = np.round(s

* LogisticRegression：0.8119

In [26]:
"""
#閾値の変更ができる。ただし今回は二値分類ですでに0/1が出ているため、意味がない

# 予測
y_proba = stacking.predict(X_test)

# 閾値の設定と調整
threshold = 0.5  # デフォルトの閾値
y_pred = (y_proba >= threshold).astype(int)

# 精度の計算
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with threshold {threshold}: {accuracy:.4f}')

# 閾値を変更して精度を再計算
threshold = 0.8
y_pred_adjusted = (y_proba >= threshold).astype(int)
accuracy_adjusted = accuracy_score(y_test, y_pred_adjusted)
print(f'Accuracy with threshold {threshold}: {accuracy_adjusted:.4f}')
"""

"\n#閾値の変更ができる。ただし今回は二値分類ですでに0/1が出ているため、意味がない\n\n# 予測\ny_proba = stacking.predict(X_test)\n\n# 閾値の設定と調整\nthreshold = 0.5  # デフォルトの閾値\ny_pred = (y_proba >= threshold).astype(int)\n\n# 精度の計算\naccuracy = accuracy_score(y_test, y_pred)\nprint(f'Accuracy with threshold {threshold}: {accuracy:.4f}')\n\n# 閾値を変更して精度を再計算\nthreshold = 0.8\ny_pred_adjusted = (y_proba >= threshold).astype(int)\naccuracy_adjusted = accuracy_score(y_test, y_pred_adjusted)\nprint(f'Accuracy with threshold {threshold}: {accuracy_adjusted:.4f}')\n"

In [27]:
"""

#SMOTE（Synthetic Minority Over-sampling Technique）は、クラス不均衡を改善するためのデータ合成技術です。
#クラス不均衡とは、データセット内のクラスの分布が偏っている状態を指します。例えば、分類問題において、クラスAが90%を占め、クラスBが10%しかない場合、これはクラス不均衡なデータセットです。
#今回は二値分類で存在確率が50％ずつであるため、不要

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from collections import Counter

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTEの適用
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 元のデータのクラス分布
print("Original dataset shape:", Counter(X_train))

# SMOTE適用後のデータのクラス分布
print("Resampled dataset shape:", Counter(X_resampled))
"""

'\n\n#SMOTE（Synthetic Minority Over-sampling Technique）は、クラス不均衡を改善するためのデータ合成技術です。\n#クラス不均衡とは、データセット内のクラスの分布が偏っている状態を指します。例えば、分類問題において、クラスAが90%を占め、クラスBが10%しかない場合、これはクラス不均衡なデータセットです。\n#今回は二値分類で存在確率が50％ずつであるため、不要\n\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.model_selection import train_test_split\nfrom collections import Counter\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# SMOTEの適用\nsmote = SMOTE(random_state=42)\nX_resampled, y_resampled = smote.fit_resample(X_train, y_train)\n\n# 元のデータのクラス分布\nprint("Original dataset shape:", Counter(X_train))\n\n# SMOTE適用後のデータのクラス分布\nprint("Resampled dataset shape:", Counter(X_resampled))\n'

In [28]:
"""
# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']


# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# モデルの構築と学習
models = [
    KNeighborsClassifier(),
    LogisticRegression(max_iter=1000),
    SVC(),
    RandomForestClassifier(),
    XGBClassifier(),
    LGBMClassifier(verbose=-1),
    DecisionTreeClassifier(),
    GradientBoostingClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    MLPClassifier(max_iter=1000)
]

# 結果を保存するためのデータフレームを作成
results = pd.DataFrame(columns=['Model', 'accuracy'])

# 各モデルに対して学習と評価を行うループ
for model in models:
    model_name = model.__class__.__name__  # モデルの名前を取得
    model.fit(X_train, y_train)  # モデルの学習
    y_pred = model.predict(X_test)  # 予測
    
    accuracy = accuracy_score(y_test, y_pred)  # 平均二乗誤差を計算
    
    # 結果をデータフレームに登録
    results = pd.concat([results, pd.DataFrame({'Model': [model_name], 'accuracy': [accuracy]})], ignore_index=True)

# MSEによって結果をソート
results = results.sort_values(by='accuracy')

# 結果を出力
display(results)
"""

'\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# モデルの構築と学習\nmodels = [\n    KNeighborsClassifier(),\n    LogisticRegression(max_iter=1000),\n    SVC(),\n    RandomForestClassifier(),\n    XGBClassifier(),\n    LGBMClassifier(verbose=-1),\n    DecisionTreeClassifier(),\n    GradientBoostingClassifier(),\n    AdaBoostClassifier(),\n    GaussianNB(),\n    MLPClassifier(max_iter=1000)\n]\n\n# 結果を保存するためのデータフレームを作成\nresults = pd.DataFrame(columns=[\'Model\', \'accuracy\'])\n\n# 各モデルに対して学習と評価を行うループ\nfor model in models:\n    model_name = model.__class__.__name__  # モデルの名前を取得\n    model.fit(X_train, y_train)  # モデルの学習\n    y_pred = model.predict(X_test)  # 予測\n    \n    accuracy = accuracy_score(y_test, y_pred)  # 平均二乗誤差を計算\n    \n    # 結果をデータフレームに登録\n    results = pd.concat([results, pd.DataFrame({\'Model\': [model_name], \'accuracy\': [accura

# Model	accuracy
* 9	GaussianNB	0.733755
* 6	DecisionTreeClassifier	0.748131
* 0	KNeighborsClassifier	0.772858
* 1	LogisticRegression	0.781484
* 8	AdaBoostClassifier	0.786084
* 10	MLPClassifier	0.787234
* 4	XGBClassifier	0.788959
* 2	SVC	0.791834
* 3	RandomForestClassifier	0.792409
* 7	GradientBoostingClassifier	0.792409
* 5	LGBMClassifier	0.806210

ダメだったこと
* 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck'　が1以上の合計値をカウントする。また、左の5つの要素を削除する
* TrueとFalseの割合がほとんど同じだった、"HomePlanet_Mars","Destination_PSO J318.5-22"の要素を削除する
* lastnameの数をカウントして、その量を特徴量とすること
* lastnameとCabin別に数値をカウントして、その量を特徴量とすること　→　ただし、model accuracyはよかった
* 歪度の調整。ShoppingMall,VRDeck,Spa,FoodCourt,VIP,RoomService,pay_sum,PassengerNum_yは横軸が伸びていたので調整
* stackingでmeta_modelをLGBM、XGBM、RFC、GBC、ABCへ変更
* stackingでcvを5から10へ変更
* stackingでtestsizeを0.05や0.4などに変更
* 外れ値を今以上に増やす。6から20ほどに増やした
* 欠損値処理をVIPごとに分けて実施。VIPであれば、VIPの中央値を補完し、VIP出ない場合は0にした
* 1人当たりの支払金額を特徴量として追加した
* HomeとDestinationの特徴量を合わせて、発着地で新たな特徴量を作成。HomeとDestinationを削除した
* VotingClassifierのsoftを利用し、アンサンブルを実施した
* 全てのカテゴリ変数をラベルエンコーディングへ変更した。destinationやHomePlanet、cabinなど。
* 外れ値の検出をもう一度行った。0.01の閾値でIsolation Forestによる外れ値検出をした
* ageとpay_sumを掛け算した、age_payという新しい特徴量を作成
* VIP,Destination,homeplanetの特徴量を削除
* LightGBMでoptunaを利用
* XGBでoptunaを利用。ただし、VIPなど関係ない特徴量も一部削除

効果が出たこと
* 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck'を0/1のカテゴリーでなく、数値で表す
* "Cabin_2"を0/1のカテゴリーでなく、数値で表す
* 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck'を合計した、"pay_sum"の要素を追加する
* Cabin_1をカテゴリーではなく、Trueの少ない順に1~8の数字を割り当てた（ただし、model accuracyではほぼ低下していたため、モデルにより使用可否を変えるべきかも）以下No2でこのmodel accuracyは表示
* StandardScalerではなく、RobustScalerでスケーリングする

# 	Model	accuracy（No2）
* 9	GaussianNB	0.714204
* 6	DecisionTreeClassifier	0.755032
* 0	KNeighborsClassifier	0.780909
* 8	AdaBoostClassifier	0.784359
* 1	LogisticRegression	0.784934
* 3	RandomForestClassifier	0.787234
* 10	MLPClassifier	0.790684
* 2	SVC	0.792984
* 7	GradientBoostingClassifier	0.794710
* 4	XGBClassifier	0.799310
* 5	LGBMClassifier	0.803910

In [29]:
"""
df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

x_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)

#define objective function for hyperparameter optimization using optuna
def objective(trial):

  #define hyperparameters to optimize for
  params = {
      'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
      'max_depth': trial.suggest_int('max_depth', 3, 10),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 1),
      'subsample': trial.suggest_uniform('subsample', 0.1, 1),
      'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
      #'gamma': trial.suggest_uniform('gamma', 0, 1),
      'alpha': trial.suggest_loguniform('alpha', 2, 5),
      'lambda': trial.suggest_loguniform('lambda', 2, 5),
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 300)
  }

  #create XGBClassifier model with optimized hyperparameters
  model = XGBClassifier(**params, random_state=0)
    
  #evaluate model using cross-validation
  score = cross_val_score(model, X, y, cv=5).mean()
    
  return score

#run hyperparameter optimization with optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
study.best_params
"""

'\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nx_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)\n\n#define objective function for hyperparameter optimization using optuna\ndef objective(trial):\n\n  #define hyperparameters to optimize for\n  params = {\n      \'n_estimators\': trial.suggest_int(\'n_estimators\', 50, 1000),\n      \'max_depth\': trial.suggest_int(\'max_depth\', 3, 10),\n      \'learning_rate\': trial.suggest_loguniform(\'learning_rate\', 0.005, 1),\n      \'subsample\': trial.suggest_uniform(\'subsample\', 0.1, 1),\n      \'colsample_bytree\': trial.suggest_uniform(\'colsample_bytree\', 0.5, 1),\n      #\'gamma\': trial.suggest_uniform(\'gamma\', 0, 1),\n      \'alpha\': trial.suggest_loguniform(\'alpha\', 2, 5),\n      \'lambda\': trial.suggest_loguniform(\'lambda\', 2, 5)

{'n_estimators': 947,
 'max_depth': 3,
 'learning_rate': 0.007935842382929551,
 'subsample': 0.9957504494825978,
 'colsample_bytree': 0.6136711694856477,
 'alpha': 3.5472522691711807,
 'lambda': 4.966323147332355,
 'min_child_weight': 17}

In [30]:
"""
df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

params = {'n_estimators': 146,
 'max_depth': 10,
 'learning_rate': 0.020428429252807186,
 'subsample': 0.4755826453445764,
 'colsample_bytree': 0.6622240313353329,
 'alpha': 2.7894408629032132,
 'lambda': 2.0202816357266684,
 'min_child_weight': 16}

model = XGBClassifier(**params, random_state=0)
model.fit(X, y)

submit = model.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)
"""

'\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nparams = {\'n_estimators\': 146,\n \'max_depth\': 10,\n \'learning_rate\': 0.020428429252807186,\n \'subsample\': 0.4755826453445764,\n \'colsample_bytree\': 0.6622240313353329,\n \'alpha\': 2.7894408629032132,\n \'lambda\': 2.0202816357266684,\n \'min_child_weight\': 16}\n\nmodel = XGBClassifier(**params, random_state=0)\nmodel.fit(X, y)\n\nsubmit = model.predict(df_test)\nsubmit = np.round(submit)\nprint(submit)\n\nsample_submission["Transported"]=submit\ndf_submit = sample_submission\ndf_submit = df_submit.replace([0,1],[False,True])\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n'

In [31]:
"""
#categorical featureなし / optunaあり

df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

x_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):

    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'max_depth':trial.suggest_int('max_depth', 1, 10),
        'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 80),
        'learning_rate':trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'verbosity': -1,
        'random_state': 0,
    }


    lgb_train = lgb.Dataset(x_train_lgb,y_train_lgb)
    lgb_valid = lgb.Dataset(x_valid_lgb,y_valid_lgb)

    gbm=lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_valid],
        valid_names=['train', 'valid'],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(100),lgb.log_evaluation(period=100)])
    
    y_prob = gbm.predict(x_valid_lgb)
    y_pred = np.round(y_prob)
    return accuracy_score(y_valid_lgb, y_pred)

study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=100)
study.best_params
"""

'\n#categorical featureなし / optunaあり\n\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nx_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)\n\ndef objective(trial):\n\n    params = {\n        \'objective\': \'binary\',\n        \'metric\': \'binary_logloss\',\n        \'boosting\': \'gbdt\',\n        \'max_depth\':trial.suggest_int(\'max_depth\', 1, 10),\n        \'min_data_in_leaf\':trial.suggest_int(\'min_data_in_leaf\', 1, 80),\n        \'learning_rate\':trial.suggest_loguniform(\'learning_rate\', 0.001, 0.1),\n        \'lambda_l1\': trial.suggest_loguniform(\'lambda_l1\', 1e-8, 10.0),\n        \'lambda_l2\': trial.suggest_loguniform(\'lambda_l2\', 1e-8, 10.0),\n        \'num_leaves\': trial.suggest_int(\'num_leaves\', 2, 512),\n        \'feature_fraction\': trial.suggest_uniform(\'feature_fra

{'max_depth': 10,<br>
 'min_data_in_leaf': 11,<br>
 'learning_rate': 0.02200392301642105,<br>
 'lambda_l1': 0.06094769921592947,<br>
 'lambda_l2': 1.844632396279779e-05,<br>
 'num_leaves': 228,<br>
 'feature_fraction': 0.8520391737889896,<br>
 'bagging_fraction': 0.9240371869522627,<br>
 'bagging_freq': 4,<br>
 'min_child_samples': 39}

In [32]:
"""
df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

x_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
'objective': 'binary',
'metric': 'binary_logloss',
'boosting': 'gbdt',
'max_depth': 10,
'min_data_in_leaf': 11,
'learning_rate': 0.02200392301642105,
'lambda_l1': 0.06094769921592947,
'lambda_l2': 1.844632396279779e-05,
'num_leaves': 228,
'feature_fraction': 0.8520391737889896,
'bagging_fraction': 0.9240371869522627,
'bagging_freq': 4,
'min_child_samples': 39,
'verbosity': -1,
'random_state': 0,
}


lgb_train = lgb.Dataset(x_train_lgb,y_train_lgb)
lgb_valid = lgb.Dataset(x_valid_lgb,y_valid_lgb)

gbm=lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['train', 'valid'],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(100),lgb.log_evaluation(period=100)])

submit = gbm.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)
"""

'\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nx_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.2, random_state=42)\n\nparams = {\n\'objective\': \'binary\',\n\'metric\': \'binary_logloss\',\n\'boosting\': \'gbdt\',\n\'max_depth\': 10,\n\'min_data_in_leaf\': 11,\n\'learning_rate\': 0.02200392301642105,\n\'lambda_l1\': 0.06094769921592947,\n\'lambda_l2\': 1.844632396279779e-05,\n\'num_leaves\': 228,\n\'feature_fraction\': 0.8520391737889896,\n\'bagging_fraction\': 0.9240371869522627,\n\'bagging_freq\': 4,\n\'min_child_samples\': 39,\n\'verbosity\': -1,\n\'random_state\': 0,\n}\n\n\nlgb_train = lgb.Dataset(x_train_lgb,y_train_lgb)\nlgb_valid = lgb.Dataset(x_valid_lgb,y_valid_lgb)\n\ngbm=lgb.train(\n    params,\n    lgb_train,\n    valid_sets=[lgb_train, lgb_valid],\n    valid_names=[\'train\', \'valid\'],\n    

In [33]:
"""
#Stratified K-Foldの活用。TransportedのTrueとFalseが既存のデータと同じ存在割合になるように、train_test_splitを行う

df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ベースモデルの定義
base_models = [
    ('knn', KNeighborsClassifier()),
    ('lr', LogisticRegression(max_iter=1000)),
    ('svc', SVC(probability=True)),
    ('rf', RandomForestClassifier()),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier(verbose=-1))
]

# メタモデルの定義
meta_model = LogisticRegression()

# Stacking Classifierの定義
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Stratified K-Foldの定義
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Stratified K-Foldクロスバリデーションの実行とモデルの学習
for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # モデルの学習
    stacking.fit(X_train_fold, y_train_fold)

    # モデルの評価
    score = stacking.score(X_test_fold, y_test_fold)
    print(f'Fold Accuracy: {score:.4f}')


submit = stacking.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)
"""

'\n#Stratified K-Foldの活用。TransportedのTrueとFalseが既存のデータと同じ存在割合になるように、train_test_splitを行う\n\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# ベースモデルの定義\nbase_models = [\n    (\'knn\', KNeighborsClassifier()),\n    (\'lr\', LogisticRegression(max_iter=1000)),\n    (\'svc\', SVC(probability=True)),\n    (\'rf\', RandomForestClassifier()),\n    (\'xgb\', XGBClassifier()),\n    (\'lgbm\', LGBMClassifier(verbose=-1))\n]\n\n# メタモデルの定義\nmeta_model = LogisticRegression()\n\n# Stacking Classifierの定義\nstacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)\n\n# Stratified K-Foldの定義\nskf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n\n# Stratified K-Foldクロスバリデーションの実行とモデルの学習\nfor train_index, test_index in skf.split

In [34]:
"""
#Voting Classifier：複数の異なる機械学習モデルの予測結果を組み合わせて、最終的な予測結果を決定するためのアンサンブル学習手法の一つ
#Soft Voting (確率平均法)とは、各モデルが予測するクラスの確率を平均し、その平均確率が最も高いクラスを最終的な予測とします。この方法では、モデルが予測した各クラスの確率の情報を利用します。

df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ベースモデルの定義
base_models = [
    ('knn', KNeighborsClassifier()),
    ('lr', LogisticRegression(max_iter=1000)),
    ('svc', SVC(probability=True)),
    ('rf', RandomForestClassifier()),
    ('xgb', XGBClassifier()),
    ('lgb', LGBMClassifier(verbose=-1))
]

# メタモデルの定義
meta_model = LogisticRegression()

# Soft Voting Classifierの定義
voting_clf_soft = VotingClassifier(estimators=base_models, voting='soft')

# Stacking Classifierの学習
voting_clf_soft.fit(X_train, y_train)
# 予測
y_pred = voting_clf_soft.predict(X_test)
# 精度の評価
accuracy = accuracy_score(y_test, y_pred)
print(f'Soft Voting Accuracy: {accuracy:.4f}')


submit = voting_clf_soft.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)
"""

'\n#Voting Classifier：複数の異なる機械学習モデルの予測結果を組み合わせて、最終的な予測結果を決定するためのアンサンブル学習手法の一つ\n#Soft Voting (確率平均法)とは、各モデルが予測するクラスの確率を平均し、その平均確率が最も高いクラスを最終的な予測とします。この方法では、モデルが予測した各クラスの確率の情報を利用します。\n\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# ベースモデルの定義\nbase_models = [\n    (\'knn\', KNeighborsClassifier()),\n    (\'lr\', LogisticRegression(max_iter=1000)),\n    (\'svc\', SVC(probability=True)),\n    (\'rf\', RandomForestClassifier()),\n    (\'xgb\', XGBClassifier()),\n    (\'lgb\', LGBMClassifier(verbose=-1))\n]\n\n# メタモデルの定義\nmeta_model = LogisticRegression()\n\n# Soft Voting Classifierの定義\nvoting_clf_soft = VotingClassifier(estimators=base_models, voting=\'soft\')\n\n# Stacking Classifierの学習\nvoting_clf_soft.fit(X_train, y_train)\n# 予測\ny_pred = voting_clf

In [35]:
"""
GradientBoostingClassifier

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

parameters = {
     "n_estimators":[i for i in range(10,100,10)],
     "learning_rate":[0.01],
     "criterion":["friedman_mse"],
     "max_depth":[i for i in range(1,5,1)],
     "min_samples_split": [2,4,5,10,12,16],
     "random_state":[3]}
#交差検証+グリッドサーチにより最良パラメータの検索
clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10)
clf.fit(X, y)

print("最良パラメータ: {}".format(clf.best_params_))
print("最良交差検証スコア: {:.2f}".format(clf.best_score_))
# # 最良パラメータ: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'max_depth': 4, 'min_samples_split': 2, 'random_state': 3}
# # 最良交差検証スコア: 0.83

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# モデルの構築と学習
model = GradientBoostingClassifier(criterion="friedman_mse", max_depth=4, min_samples_split=2, n_estimators=90, learning_rate=0.01, random_state=3)
#最良パラメータ: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'max_depth': 4, 'min_samples_split': 2, 'n_estimators': 90, 'random_state': 3}
#最良交差検証スコア: 0.74

# 各モデルに対して学習と評価を行うループ
model.fit(X_train, y_train)  # モデルの学習
y_pred = model.predict(X_test)  # 予測

accuracy = accuracy_score(y_test, y_pred)  # 平均二乗誤差を計算
print(accuracy)

submit = model.predict(df_test)
print(submit)
sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

"""

'\nGradientBoostingClassifier\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nparameters = {\n     "n_estimators":[i for i in range(10,100,10)],\n     "learning_rate":[0.01],\n     "criterion":["friedman_mse"],\n     "max_depth":[i for i in range(1,5,1)],\n     "min_samples_split": [2,4,5,10,12,16],\n     "random_state":[3]}\n#交差検証+グリッドサーチにより最良パラメータの検索\nclf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10)\nclf.fit(X, y)\n\nprint("最良パラメータ: {}".format(clf.best_params_))\nprint("最良交差検証スコア: {:.2f}".format(clf.best_score_))\n# # 最良パラメータ: {\'criterion\': \'friedman_mse\', \'learning_rate\': 0.01, \'max_depth\': 4, \'min_samples_split\': 2, \'random_state\': 3}\n# # 最良交差検証スコア: 0.83\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# モデルの構築と学習\nmodel = GradientBoostingClassifier(criterion=

In [36]:
"""
df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

lre = LogisticRegression()
param_test = {'C':[0.001,0.01,0.1,1,10,100,1000]}
#Cとは、「コストパラメーター」の略であり、誤分類を許容する指標のこと。Cが大きいと誤分類を許容しない（外れ値をいれて評価）ため、学習されやすくなる。Cが小さすぎると誤分類を許容する（外れ値をいれずに評価）ため、学習が進みにくい。基本は1である

grid = GridSearchCV(estimator = lre, param_grid = param_test, scoring='accuracy', cv=10)
grid.fit(X, y)
print(grid.best_params_, grid.best_score_, sep="\n")


submit = model.predict(df_test)
print(submit)
sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

"""

'\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\nlre = LogisticRegression()\nparam_test = {\'C\':[0.001,0.01,0.1,1,10,100,1000]}\n#Cとは、「コストパラメーター」の略であり、誤分類を許容する指標のこと。Cが大きいと誤分類を許容しない（外れ値をいれて評価）ため、学習されやすくなる。Cが小さすぎると誤分類を許容する（外れ値をいれずに評価）ため、学習が進みにくい。基本は1である\n\ngrid = GridSearchCV(estimator = lre, param_grid = param_test, scoring=\'accuracy\', cv=10)\ngrid.fit(X, y)\nprint(grid.best_params_, grid.best_score_, sep="\n")\n\n\nsubmit = model.predict(df_test)\nprint(submit)\nsample_submission["Transported"]=submit\ndf_submit = sample_submission\ndf_submit = df_submit.replace([0,1],[False,True])\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n'

In [37]:

#Lightgbm

df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

X = df_train.drop("Transported", axis=1)
X = X[selected_columns]
y = df_train['Transported']

x_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(X, y, test_size=0.4, random_state=0)

params = {'objective': 'binary',
          'metric': 'binary_logloss',
          'boosting': 'gbdt',
          'verbosity': -1,
          'random_state': 1}

lgb_train = lgb.Dataset(x_train_lgb,y_train_lgb)
lgb_valid = lgb.Dataset(x_valid_lgb,y_valid_lgb)

gbm=lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['train', 'valid'],
    num_boost_round=5000,
    callbacks=[lgb.early_stopping(200),lgb.log_evaluation(period=100)])

y_prob = gbm.predict(x_valid_lgb)
y_pred = np.round(y_prob)
print(accuracy_score(y_valid_lgb, y_pred))

df_test = df_test[selected_columns]
submit = gbm.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"]=submit
df_submit = sample_submission
df_submit = df_submit.replace([0,1],[False,True])
df_submit.to_csv("/kaggle/working/submission.csv",index=False)


Training until validation scores don't improve for 200 rounds
[100]	train's binary_logloss: 0.282784	valid's binary_logloss: 0.435194
[200]	train's binary_logloss: 0.230351	valid's binary_logloss: 0.456092
Early stopping, best iteration is:
[45]	train's binary_logloss: 0.339771	valid's binary_logloss: 0.428847
0.7905035971223021
[1. 0. 1. ... 1. 1. 1.]


In [38]:
"""
#ハイパーパラメータの調整を行ったstacking

df_submit = pd.DataFrame()
df_submit.to_csv("/kaggle/working/submission.csv",index=False)

# 説明変数と目的変数を選択
X = df_train.drop("Transported", axis=1)
y = df_train['Transported']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear']
}

param_grid_svc = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9]
}

param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 62, 127]
}

knn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=1000)
svc = SVC(probability=True)
rf = RandomForestClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()


#ハイパーパラメータをgridsearchで調整。これを入れると長くなるため、パラメーターだけ抽出して以下へ記載

models_param_grid = [
    (knn, param_grid_knn),
    (lr, param_grid_lr),
    (rf, param_grid_rf),
    (xgb, param_grid_xgb),
    (lgbm, param_grid_lgbm)
]

# Perform GridSearchCV for each model
best_estimators = []
for model, param_grid in models_param_grid:
    print(model)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_estimators.append((model.__class__.__name__, grid_search.best_estimator_))

# Output the best estimators
for name, estimator in best_estimators:
    print(f'Best estimator for {name}: {estimator}')

#ハイパーパラメータの一覧
KNeighborsClassifier()
LogisticRegression(max_iter=1000)
RandomForestClassifier()
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)
LGBMClassifier(verbose=-1)
Best estimator for KNeighborsClassifier: KNeighborsClassifier(metric='manhattan', n_neighbors=7)
Best estimator for LogisticRegression: LogisticRegression(C=1, max_iter=1000, solver='liblinear')
Best estimator for RandomForestClassifier: RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=200)
Best estimator for XGBClassifier: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)
Best estimator for LGBMClassifier: LGBMClassifier(verbose=-1)    
    
base_models = [
    ('knn', KNeighborsClassifier(metric='manhattan', n_neighbors=7)),
    ('lr', LogisticRegression(C=1, max_iter=1000, solver='liblinear')),
    ('svc', SVC(probability=True)),
    ('rf', RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=200)),
    ('xgb', XGBClassifier(learning_rate=0.2,max_depth=3,n_estimators=200)),
    ('lgbm', LGBMClassifier(verbose=-1))
]

meta_model = LogisticRegression()

stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = stacking_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Stacking Model Accuracy: {accuracy:.4f}')

# Predict on test data
submit = stacking_clf.predict(df_test)
submit = np.round(submit)
print(submit)

sample_submission["Transported"] = submit
sample_submission = sample_submission.replace([0, 1], [False, True])
sample_submission.to_csv("/kaggle/working/submission.csv", index=False)
    
"""

'\n#ハイパーパラメータの調整を行ったstacking\n\ndf_submit = pd.DataFrame()\ndf_submit.to_csv("/kaggle/working/submission.csv",index=False)\n\n# 説明変数と目的変数を選択\nX = df_train.drop("Transported", axis=1)\ny = df_train[\'Transported\']\n\n# データの分割\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\nparam_grid_knn = {\n    \'n_neighbors\': [3, 5, 7],\n    \'weights\': [\'uniform\', \'distance\'],\n    \'metric\': [\'euclidean\', \'manhattan\']\n}\n\nparam_grid_lr = {\n    \'C\': [0.1, 1, 10, 100],\n    \'solver\': [\'newton-cg\', \'lbfgs\', \'liblinear\']\n}\n\nparam_grid_svc = {\n    \'C\': [0.1, 1, 10, 100],\n    \'gamma\': [1, 0.1, 0.01, 0.001],\n    \'kernel\': [\'rbf\', \'linear\']\n}\n\nparam_grid_rf = {\n    \'n_estimators\': [100, 200, 300],\n    \'max_depth\': [None, 10, 20, 30],\n    \'min_samples_split\': [2, 5, 10]\n}\n\nparam_grid_xgb = {\n    \'n_estimators\': [100, 200, 300],\n    \'learning_rate\': [0.01, 0.1, 0.2],\n    \'max_depth\': [3, 6, 9]\n}\n\

In [39]:
"""
import shap

# SHAP値の計算
explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(X_train)

# SHAP値のプロット
shap.summary_plot(shap_values, X_train)
"""

'\nimport shap\n\n# SHAP値の計算\nexplainer = shap.TreeExplainer(gbm)\nshap_values = explainer.shap_values(X_train)\n\n# SHAP値のプロット\nshap.summary_plot(shap_values, X_train)\n'